# Create an aircraft from scratch

In this tutorial we will
 - Open a simple baseline CPACS file with TiXI and TiGL
 - Add sections to the fuselage to model a nose and tail
 - Add a horizontal tailplane (HTP) to the configuration
 - Export the configuration to a new CPACS file.

## Open a baseline CPACS file with TiXI and TiGL

We start by importing the required modules. We will use `tixi3wrapper` and `tigl3wrapper` to open the CPACS file and we will use `CCPACSConfigurationManager_get_instance` to obtain an instance of TiGL's configuration manager. This class is used to access the internal Python API.

In [1]:
from tixi3 import tixi3wrapper
from tigl3 import tigl3wrapper
from tigl3.configuration import CCPACSConfigurationManager_get_instance
from tigl3.geometry import CTiglPoint, TIGL_X_Z_PLANE 
import os
import sys

Let's open the base line configuration!

In [2]:
baseline = os.path.join("..", "data", "simpletest.cpacs.xml")

# create handles
tixi_h = tixi3wrapper.Tixi3()
tigl_h = tigl3wrapper.Tigl3()

#initialize handles and open the CPACS file
tixi_h.open(baseline)
tigl_h.open(tixi_h, "") 

Let's get an instance of the configuration manager and access the CPACS configuration

In [3]:
 # To access the internal python api, you need to use the configuration manager
mgr =  CCPACSConfigurationManager_get_instance()
aircraft = mgr.get_configuration(tigl_h._handle.value)

To visualize the baseline aircraft, we will use a pre-written display function from `helper.py`. 

Feel free to have a peek, but the implementation is irrelevant for this turorial

In [4]:
# lets add the parent directory to the path and import the display function from the helper module
sys.path.insert(1, "..")
from helper import display

# Show it to me!
display(aircraft)

### Add sections to the fuselage to model a nose and tail

Our next task will be to modify the shape of the fuselage. 

Let's begin by adding a nose section (element) before the first using the function `fuselage.create_new_connected_element_before(element_uid, new_uid)`.

*Hint:* The first element of the fuselage has the UID "D150_Fuselage_1Section1IDElement1".

In [5]:
fuselage = aircraft.get_fuselage(1)

# add nose
fuselage.create_new_connected_element_before("D150_Fuselage_1Section1IDElement1", "nose")

Now we will scale the first section to a point by setting the area to zero.

In [6]:
s = fuselage.get_section(1)
e = s.get_section_element(1)
ce = e.get_ctigl_section_element()

# set the area of the section element to zero
ce.set_area(0)

Now we will modify the tail of the fuselage. We create a new section after the last one, analogously to the previous task.

*Hint:* The last element of the fuselage has the UID "D150_Fuselage_1Section3IDElement1".

In [7]:
# add tail
fuselage.create_new_connected_element_after("D150_Fuselage_1Section3IDElement1", "tail")

We will translate the section and scale it.

*Hint:* You need to use the class `CTiglPoint` here.

In [8]:
s = fuselage.get_section(fuselage.get_section_count()) # This should have the index 5
e = s.get_section_element(1)
ce = e.get_ctigl_section_element()

# set the center of the section element to (x,y,z) = (3.55, 0., 0.3)
ce.set_center(CTiglPoint(3.55, 0., 0.3))

# set the area of the section element to 0.05
ce.set_area(0.05)

It's time to have a look at what we have so far!

In [9]:
display(aircraft)

### Add a HTP to the configuration

Now we will add a new wing. We use the `CCPACSWing::create_wing` method. 

We need to provide a UID for the new wing, the number of sections as well as the UID of the airfoil we want to use for each section. 

We use
 - `wingUID="htp"`
 - `numberOfSections=3`
 - `airfoilUID="NACA0012"`

In [10]:
# create the htp
wings = aircraft.get_wings()
htp = wings.create_wing(wingUID="htp", numberOfSection=3, airfoilUID="NACA0012")

Now we need to sculpt the horizontal tailplane to our needs.

*Hint:* Use the python help function `help(htp)` to get an overview of available methods.

Your task is to 
 - set the area of the HTP to 0.5 while maintaining the aspect ratio
 - set the root Leading Edge (LE) position to (x,y,z) = (3.0, 0., 0.3)
 - set the symmetry to `TIGL_X_Z_PLANE`

In [11]:
# modify htp parameters
htp.set_area_keep_ar(0.5)
htp.set_root_leposition(CTiglPoint(3.0, 0., 0.3))
htp.set_symmetry(TIGL_X_Z_PLANE)

After having set some global parameters on the HTP, let's fine tune the area and translation of the outer section.

In [12]:
s = htp.get_section(3)
e = s.get_section_element(1)
ce = e.get_ctigl_section_element()

# set the center of the section element ce to (x,y,z) = (3.4, 1., 0.3)
ce.set_center(CTiglPoint(3.4, 1., 0.3))

# set the area of the section element ce to 0.01
ce.set_area(0.01)

What do we have so far?

In [13]:
display(aircraft)

### Export the configuration to a new CPACS file.

We are done! We still need to save our results. 

First, we need to write our modifications to CPACS. This updates the underlying TiXI handle. Then we can save the document.

In [14]:
aircraft.write_cpacs(aircraft.get_uid())
outfile = "my_awesome_aircraft.xml"
tixi_h.save(outfile)

Have a look at the CPACS file. Would you have wanted to write this by hand?